In [11]:
from VAE import VAE
import sys

sys.path.append("../../")
from utils import load_csv, dataloader, load_save_models
import torch
from torchvision.transforms import v2

In [12]:
batch_size = 436
input_dim = 4800
hidden_dim = 4000
latent_dim = 320

In [13]:
model = VAE(input_dim,hidden_dim,latent_dim)
VAE.load_state_dict(model,state_dict=torch.load('./vae.pth'))

/tmp/ipykernel_34806/2905130577.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  VAE.load_state_dict(model,state_dict=torch.load('./vae.pth'))


<All keys matched successfully>

# load in data necessary for computing the threshold

In [14]:
set = load_csv.load_pandas()
train,val,test = load_csv.split_data(set)

/home/bendm/machine_learning/project/majority-vote-outlier-detection/models/VAE/../../utils/load_csv.py:20: DtypeWarning: Columns (91,94,209,213) have mixed types. Specify dtype option on import or set low_memory=False.
  frame = pd.read_csv(data_path, delimiter=";").iloc[:, -3:]


In [ ]:
transform = v2.Compose(
    [
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Resize((60, 80)),
        v2.ToTensor(),
        v2.Lambda(
            lambda x: (x.view(-1) - torch.min(x)) / (torch.max(x) - torch.min(x))
        ),
    ]
)

val_set = dataloader(train, transform)

val_dataloader = torch.utils.data.DataLoader(
    val_set, batch_size=batch_size, shuffle=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

